In [ ]:
###
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC 
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn import model_selection
import pandas as pd
from sklearn.preprocessing import StandardScaler

linnarson_DUB = pd.read_csv('data/linnarson_sub_DUB.csv',index_col=0)
all_data_DUB = pd.read_csv('data/harmonized_all_data_sub_DUB.csv',index_col=0)

base = linnarson_DUB.drop('cell_type',axis = 1)

labels = linnarson_DUB['cell_type'].values

base = base[base.columns & all_data_DUB.columns]
base = base.loc[:,all_data_DUB.columns.values.tolist()]

scaler = StandardScaler()

base_scaled = scaler.fit_transform(base)

In [ ]:
### Gridseacrch

X_train, X_test, y_train, y_test = train_test_split(base_scaled, labels,stratify=labels, test_size=0.2,shuffle=True)

# Define classifiers
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    GaussianNB(),
    SVC(),
    MLPClassifier()
]

# Define hyperparameter grids for each classifier
param_grids = [
    {'n_neighbors': [3, 5, 7, 10, 12, 15, 17, 20]},
    {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]},
    {},
    {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid']},
    {'hidden_layer_sizes': [(532,2)],
      'activation': ['tanh','relu'],
     'max_iter' : [450,500,600],
     'solver' : ['sgd','adam'],
    'learning_rate_init': [0.001,0.01,0.1]}
]

class_labels = np.unique(labels)
best_models = []
# Perform grid search for each classifier
for classifier, param_grid in zip(classifiers, param_grids):
    grid_search = GridSearchCV(classifier, param_grid, cv=3, n_jobs=-1, scoring='f1_macro')
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    test_f1 = f1_score(y_test, y_pred, average='macro')
    test_recall = recall_score(y_test, y_pred, average='macro')
    test_accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    print("Classifier:", classifier.__class__.__name__)
    print("Best Parameters:", grid_search.best_params_)
    print("Best F1-score:", grid_search.best_score_)
    print("Test F1-score:", test_f1)
    print("Test Recall:", test_recall)
    print("Test Accuracy:", test_accuracy)
    print("Confusion Matrix:")
    print("Class Labels:", class_labels)
    for idx, label in enumerate(class_labels):
        print(f"{label:<10}", end="")
        for value in conf_matrix[idx]:
            print(f"{value:<10}", end="")
        print()
    
    print("------------------------")
    best_models.append(best_model)


In [ ]:
# Classify the new dataset using the best models


all_data_DUB_scaled = scaler.fit_transform(all_data_DUB)

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    GaussianNB(),
    SVC(),
    MLPClassifier()
]

models = [

    KNeighborsClassifier(n_neighbors=10),
    DecisionTreeClassifier(max_depth=10,min_samples_split=5),
    GaussianNB(priors=None, var_smoothing=1e-9),
    SVC(kernel='rbf'),
    MLPClassifier(hidden_layer_sizes=(532,2), activation='relu',max_iter=450,solver='adam',learning_rate_init=0.01)
]

prediction = []
for classifier, model in zip(classifiers, models):
    model_predictions = model.fit(base_scaled,labels)
    y_pred = model_predictions.predict(all_data_DUB_scaled)
    prediction.append(y_pred)
    
    print("Classifier:", classifier.__class__.__name__)
    print("------------------------ Done")

In [ ]:
all_data_DUB['kNN'] = prediction[0].tolist()
all_data_DUB['DT'] = prediction[1].tolist()
all_data_DUB['NB'] = prediction[2].tolist()
all_data_DUB['SVM'] = prediction[3].tolist()
all_data_DUB['MLP'] = prediction[4].tolist()

In [ ]:
all_data_DUB.to_csv('data/all_data_DUB_grid_predicted.csv')